## Set up google colab environment

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [3]:
from SGCNN_12_Utils import *
import scipy.io as sio

## Load Indian Pines Dataset - Source

In [4]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [5]:
data_source = uIndianPines['indian_pines_corrected']
ground_truth_source = gt_IndianPines['indian_pines_gt']

In [6]:
data_source.shape

(145, 145, 200)

In [7]:
ground_truth_source.shape

(145, 145)

# Load target dataset

In [8]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [9]:
data_target = uBotswana['Botswana']
ground_truth_target = gt_Botswana['Botswana_gt']

In [10]:
data_target.shape

(1476, 256, 145)

In [11]:
ground_truth_target.shape

(1476, 256)

## Distrubution of samples for each class in Source

In [12]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [13]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_source

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

# Class distribution of samples in Target

In [14]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [15]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[1:] ## Dropping classes with small number of samples
classes_target

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with different overlap ratios and a percent of samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [16]:
pretrain_results = pretrain_source_models(percentages = [60,70,80,90],
                                          classes = classes_source,
                                          cube_size = 20,
                                          overlap_ratios = [0.8,0.9,1],
                                          data = data_source,
                                          ground_truth = ground_truth_source,
                                          batch_size = 20,
                                          channels = 64,
                                          epochs = 100,
                                          Verbosity = 1,
                                          accuracies = [],
                                          learning_rate = 0.0001,
                                          source_dataset = 'indian_pines')

Streaming output truncated to the last 5000 lines.
conv2d_240 (Conv2D)             (None, 20, 20, 8)    584         conv2d_239[0][0]                 
__________________________________________________________________________________________________
conv2d_241 (Conv2D)             (None, 20, 20, 8)    584         lambda_102[0][0]                 
__________________________________________________________________________________________________
conv2d_244 (Conv2D)             (None, 20, 20, 8)    584         conv2d_243[0][0]                 
__________________________________________________________________________________________________
concatenate_12 (Concatenate)    (None, 20, 20, 64)   0           conv2d_229[0][0]                 
                                                                 conv2d_232[0][0]                 
                                                                 conv2d_233[0][0]                 
                                                          

In [17]:
pretrain_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,477,323,60,65.94
1,0.8,555,245,70,68.57
2,0.8,636,164,80,75.61
3,0.8,715,85,90,70.59
4,0.9,2348,1570,60,77.83
5,0.9,2740,1178,70,75.98
6,0.9,3132,786,80,79.13
7,0.9,3523,395,90,84.56
8,1.0,4740,3166,60,77.51
9,1.0,5530,2376,70,71.93


# Fine tune on botswana

In [18]:
transfer_results, confusion_matrixes = transfer_learning(percentages = [60,70,80,90],
                                                        source_dataset = 'indian_pines',
                                                        target_dataset = 'botswana',
                                                        data = data_target,
                                                        ground_truth = ground_truth_target,
                                                        classes = classes_target,
                                                        overlap_ratios = [0.8,0.9,1],
                                                        channels = 64,
                                                        cube_size = 20,
                                                        learning_rate = 0.0001,
                                                        epochs = 100,
                                                        batch_size = 20)

Streaming output truncated to the last 5000 lines.
__________________________________________________________________________________________________
Samples per class: [136, 52, 125, 110, 132, 128, 125, 97, 145, 120, 131, 68, 110, 50]
Total number of samples is 1529.

unique classes in training set: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Total number of samples in training set is 914.
Samples per class in training set: [81 31 75 66 79 76 75 58 87 72 78 40 66 30]

unique classes in test set: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Total number of samples in test set is 615.
Samples per class in test set: [55 21 50 44 53 52 50 39 58 48 53 28 44 20]

X_train_transfer => (914, 256)
X_test_transfer  => (615, 256)
y_train => (914, 14)
y_test  => (615, 14)

Model: "fine_tune"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 256)]             0         
____________

In [19]:
transfer_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,251,175,60,74.86
1,0.8,293,133,70,87.22
2,0.8,337,89,80,92.13
3,0.8,378,48,90,95.83
4,0.9,914,615,60,86.18
5,0.9,1065,464,70,88.36
6,0.9,1219,310,80,95.16
7,0.9,1371,158,90,95.57
8,1.0,1880,1264,60,87.50
9,1.0,2195,949,70,93.99


In [20]:
for cm in confusion_matrixes:
  print(cm)

                1  2   3   4   5  ...  11  12  13  14  classfication_accuracies
0              13  0   0   0   0  ...   0   0   0   0                     92.86
1               0  5   0   0   1  ...   0   0   0   0                     83.33
2               0  0   7   0   0  ...   0   0   0   0                      50.0
3               0  0   0  11   0  ...   0   0   0   0                     91.67
4               3  0   0   2  17  ...   0   0   0   0                     70.83
5               0  0   0   0   0  ...   0   0   0   0                       0.0
6               3  0   0   0   0  ...   0   0   0   0                     76.92
7               0  0   0   0   0  ...   0   1   0   0                      90.0
8               0  0   0   0   0  ...   0   0   0   0                     100.0
9               0  0   0   0   0  ...   0   0   0   0                     100.0
10              0  0   0   0   0  ...  10   0   0   0                     76.92
11              0  0   0   0   0  ...   